In [1]:
# imports
import hashlib
import pandas as pd
from cryptography.hazmat.primitives import hashes
import sys
import time
from cryptography.hazmat.primitives import serialization
#import struct
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.serialization import load_der_private_key
from cryptography.hazmat.primitives.serialization import load_der_public_key
from cryptography.hazmat.primitives.asymmetric import utils, ec
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.serialization import Encoding
from cryptography.hazmat.primitives.serialization import PublicFormat
from cryptography.hazmat.primitives import hashes

In [2]:
# table for program and essay weighting
table = pd.DataFrame({'Program':[70,50,30],
                      'Essay':[30,50,70],
                      'Tick one box below':['x','','']})
display(table)

,Program,Essay,Tick one box below
0,70,30,x
1,50,50,
2,30,70,


In [3]:
# transaction class with parameters
class Transaction:
    def __init__(self, sender_hash, recipient_hash, sender_public_key, amount, fee, nonce, signature, txid):            
        self.sender_hash = sender_hash
        self.recipient_hash = recipient_hash
        # if public key is in bytes,like in the last testing exercise, it will be deserialized
        if isinstance(sender_public_key, bytes):
            self.sender_public_key = load_der_public_key(sender_public_key)
        else:
            self.sender_public_key = sender_public_key
        self.amount = amount
        self.fee = fee
        self.nonce = nonce
        self.signature = signature
        self.txid = txid
        
    def verify(self, sender_balance, sender_previous_nonce):
        if not (len(self.sender_hash) == len(self.recipient_hash) == 20):
            raise ValueError("Hash must be of size 20!")
        if not (self.sender_hash == calculate_sha1_hash(self.sender_public_key)):
            raise ValueError("Sender hash does not match the sender's public key!")
        if not (1 <= self.amount <= sender_balance):
            raise ValueError("Invalid amount!")
        if not (0 <= self.fee <= self.amount):
            raise ValueError("Invalid fee!")
        if not (self.nonce == sender_previous_nonce + 1):
            raise ValueError("Invalid nonce!")
        if not (self.txid == calculate_txid(
                    self.sender_hash, 
                    self.recipient_hash, 
                    self.sender_public_key, 
                    self.amount, 
                    self.fee, 
                    self.nonce, 
                    self.signature)):
            raise ValueError("Invalid transaction ID!")
        signature_hash = calculate_signature_hash(
                    self.recipient_hash,
                    self.amount,
                    self.fee,
                    self.nonce)
        self.sender_public_key.verify(self.signature, signature_hash,
                ec.ECDSA(utils.Prehashed(hashes.SHA256())))
        
        
def calculate_sha1_hash(public_key):
    digest = hashes.Hash(hashes.SHA1(),default_backend())
    digest.update(encode_public_key(public_key))
    return digest.finalize()


def calculate_txid(sender_hash, recipient_hash, sender_public_key, amount, fee, nonce, signature):
    m = hashes.Hash(hashes.SHA256(),default_backend())
    m.update(sender_hash)
    m.update(recipient_hash)
    m.update(encode_public_key(sender_public_key))
    m.update((amount).to_bytes(8, byteorder = 'little', signed = False))
    m.update((fee).to_bytes(8, byteorder = 'little', signed = False))
    m.update((nonce).to_bytes(8, byteorder = 'little', signed = False))
    # initially used the struct library for little endian representation
    #m.update(struct.pack("<Q", amount))
    #m.update(struct.pack("<Q", fee))
    #m.update(struct.pack("<Q", nonce))
    m.update(signature)
    return m.finalize()  

def calculate_signature_hash(recipient_hash, amount, fee, nonce):
    m = hashes.Hash(hashes.SHA256(),default_backend())
    m.update(recipient_hash)
    m.update((amount).to_bytes(8, byteorder = 'little', signed = False))
    m.update((fee).to_bytes(8, byteorder = 'little', signed = False))
    m.update((nonce).to_bytes(8, byteorder = 'little', signed = False))
    return m.finalize()

def private_key_to_public_key(private_key):
    public_key = private_key.public_key()
    return public_key


def encode_public_key(public_key):
    return public_key.public_bytes(encoding=Encoding.DER,format=PublicFormat.SubjectPublicKeyInfo)


def create_signed_transaction(sender_private_key, recipient_hash, amount, fee, nonce):
    sender_public_key = private_key_to_public_key(sender_private_key)
    sender_hash = calculate_sha1_hash(sender_public_key)
    signature_hash = calculate_signature_hash(recipient_hash, amount, fee, nonce)
    signature = sender_private_key.sign(signature_hash, ec.ECDSA(utils.Prehashed(hashes.SHA256())))
    txid = calculate_txid(sender_hash, recipient_hash, sender_public_key, amount, fee, nonce, signature)
    return Transaction(
            sender_hash, 
            recipient_hash, 
            sender_public_key, 
            amount, 
            fee, 
            nonce, 
            signature, 
            txid)

### Testing

#### 1.Testcase
Generate a private key using ec.generate_private_key(ec.SECP256K1) . Call
create_signed_transaction to make a test transaction. Check that the
transaction.verify call succeeds (when provided with a sender_balance which is
sufficiently high and sender_previous_nonce = transaction.nonce - 1 ).

In [4]:
# testing

# creating keys
sender_private_key = ec.generate_private_key(ec.SECP256K1)
recipient_private_key = ec.generate_private_key(ec.SECP256K1)
recipient_public_key = private_key_to_public_key(recipient_private_key)
recipient_hash = calculate_sha1_hash(recipient_public_key)

In [5]:
# generate valid transaction
transaction_testcase1 = create_signed_transaction(sender_private_key, recipient_hash, 4, 1, 2)

In [6]:
transaction_testcase1.verify(100, 1)

####  1.Testcase Sucess!


#### 2.Testcase 

Generate a valid transaction, check that modifying any of the fields causes
transaction.verify to raise an exception due to an invalid txid .

In [7]:
transaction_testcase2 = create_signed_transaction(sender_private_key, recipient_hash, 4, 1, 2)

In [8]:
# modifying any of the fields in this case fee
transaction_testcase2.fee = 3

In [9]:
transaction_testcase2.verify(100,1)

ValueError: Invalid transaction ID!

####  2.Testcase Sucess!

#### 3.Testcase

Generate a valid transaction, change the amount field, regenerate the txid so it is valid
again. Check that transaction.verify raises an exception due to an invalid signature.

In [10]:
# generate valid transaction
transaction_testcase3 = create_signed_transaction(sender_private_key, recipient_hash, 4, 1, 2)

In [11]:
# changeing the amount field
transaction_testcase3.amount = 3

In [12]:
# regenerating txid so its valid again
transaction_testcase3.txid = calculate_txid(transaction_testcase3.sender_hash,
                                  transaction_testcase3.recipient_hash,
                                  transaction_testcase3.sender_public_key,
                                  transaction_testcase3.amount,
                                  transaction_testcase3.fee,
                                  transaction_testcase3.nonce,
                                  transaction_testcase3.signature)

In [13]:
# check if transaction verify rasies an exception due to invald signature
transaction_testcase3.verify(4,1)

InvalidSignature: 

####  3.Testcase Sucess!

#### 4.Testcase

Generate a valid transaction, check that transaction.verify raises an exception if either
the sender_balance is too low or sender_previous_nonce is incorrect.

In [14]:
# creating valid transaction
transaction_testcase4 = create_signed_transaction(sender_private_key, recipient_hash, 10, 2, 1)

In [15]:
# check if transaction verify raises an exception if sender_previous_nonce is incorrect
transaction_testcase4.verify(10,2)

ValueError: Invalid nonce!

In [16]:
# check if transaction verify raises an exception if sender_balance is incorrect
transaction_testcase4.verify(9,0)

ValueError: Invalid amount!

####  4.Testcase Sucess!

#### 5.Testcase

Generate two private keys, A and B . Use A to generate a valid transaction. Replace the
signature with a signature created using B . Regenerate the txid and confirm that
transaction.verify fails with an invalid signature.

In [17]:
# generating transactions for priavte key A and B
sender_private_key_a = ec.generate_private_key(ec.SECP256K1)
sender_private_key_b = ec.generate_private_key(ec.SECP256K1)

recipient_private_key = ec.generate_private_key(ec.SECP256K1)
recipient_private_key = ec.generate_private_key(ec.SECP256K1)
recipient_public_key = private_key_to_public_key(recipient_private_key)
recipient_hash = calculate_sha1_hash(recipient_public_key)

transaction_a = create_signed_transaction(sender_private_key_a, recipient_hash, 10, 2, 1)
transaction_b = create_signed_transaction(sender_private_key_b, recipient_hash, 10, 2, 1)

In [18]:
# replace signature of transaction_a with signature of transaction b
transaction_a.signature = transaction_b.signature

In [19]:
# regenerating the txid 
transaction_a.txid = calculate_txid(transaction_a.sender_hash,
                                    transaction_a.recipient_hash,
                                    transaction_a.sender_public_key,
                                    transaction_a.amount,
                                    transaction_a.fee,
                                    transaction_a.nonce,
                                    transaction_a.signature)

In [20]:
# confirming that transaction verify fails with an invald signature
transaction_a.verify(10,0)

InvalidSignature: 

####  5.Testcase Sucess!

#### 6.Testcase

Check that the following transaction verifies successfully (when using sender_balance = 20 ,
sender_previous_nonce = 4 )

In [21]:
transaction = Transaction(bytes.fromhex("3df8f04b3c159fdc6631c4b8b0874940344d173d"),
            bytes.fromhex("5c1499a0484ace2f731b0afb83241e15f0e168ca"),
            bytes.fromhex("3056301006072a8648ce3d020106052b8104000a" +
                          "03420004886ed03cb7ffd4cbd95579ea2e202f1d" +
                          "b29afc3bf5d7c2c34a34701bbb0685a7b535f1e6" +
                          "31373afe8d1c860a9ac47d8e2659b74d437435b0" +
                          "5f2c55bf3f033ac1"
                         ),
            10,
            2,
            5,
            bytes.fromhex("3046022100f9c076a72a2341a1b8cb68520713e1" +
                          "2f173378cf78cf79c7978a2337fbad141d022100" +
                          "ec27704d4d604f839f99e62c02e65bf60cc93ae1" +
                          "735c1ccf29fd31bd3c5a40ed"),
            bytes.fromhex("ca388e0890b71bd1775460d478f26af3776c9b4f" + 
                          "6c2b936e1e788c5c87657bc3")
           
           )

In [22]:
transaction.verify(20,4)

#### 6.Testcase Sucess!